In [13]:
import pandas as pd 
import numpy as np

from  datetime import datetime, timedelta
import gc
import numpy as np, pandas as pd
import lightgbm as lgb


In [2]:
import os
for dirname, _, filenames in os.walk('C:\\Eric\\Data\\M5'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

C:\Eric\Data\M5\SNAP_CA.csv
C:\Eric\Data\M5\SNAP_TX.csv
C:\Eric\Data\M5\SNAP_WI.csv


In [3]:
SNAP_pl_ca = pd.read_csv("C:\Eric\Data\M5\SNAP_CA.csv")
SNAP_pl_tx = pd.read_csv("C:\Eric\Data\M5\SNAP_TX.csv")
SNAP_pl_wi = pd.read_csv("C:\Eric\Data\M5\SNAP_WI.csv")

In [6]:
SNAP_pl_ca.dtypes

DATE              object
BRCA06M647NCEN     int64
dtype: object

In [7]:
import os
for dirname, _, filenames in os.walk('C:\Eric\Data\m5-forecasting-accuracy'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

C:\Eric\Data\m5-forecasting-accuracy\calendar.csv
C:\Eric\Data\m5-forecasting-accuracy\M5_codes.txt
C:\Eric\Data\m5-forecasting-accuracy\sales_train_evaluation.csv
C:\Eric\Data\m5-forecasting-accuracy\sales_train_validation.csv
C:\Eric\Data\m5-forecasting-accuracy\sample_submission.csv
C:\Eric\Data\m5-forecasting-accuracy\sell_prices.csv


In [8]:
calendar         = pd.read_csv("C:\Eric\Data\m5-forecasting-accuracy\calendar.csv")
train_validation = pd.read_csv("C:\Eric\Data\m5-forecasting-accuracy\sales_train_validation.csv")
sell_prices = pd.read_csv("C:\Eric\Data\m5-forecasting-accuracy\sell_prices.csv")

In [10]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [11]:
calendar = reduce_mem_usage(calendar)
train_validation = reduce_mem_usage(train_validation)
sell_prices = reduce_mem_usage(sell_prices)

Memory usage after optimization is: 0.12 MB
Decreased by 41.9%
Memory usage after optimization is: 95.00 MB
Decreased by 78.7%
Memory usage after optimization is: 130.48 MB
Decreased by 37.5%


In [12]:
SNAP_pl_ca.head()

,DATE,BRCA06M647NCEN
0,1981-01-01,1597227
1,1981-02-01,1615981
2,1981-03-01,1645020
3,1981-04-01,1635941
4,1981-05-01,1612883


In [ ]:
# snap col 이름 수정, [date, number of people]
# train 에 붙이기 ... !  train 에 date 랑 맞춰서 ..  월간 데이터니까..  month 만 분리해서.. 맞는거에 집어넣기..  


In [20]:
calendar['snap_CA'].unique()

array([0, 1], dtype=int64)

In [26]:
calendar.columns

Index(['date', 'wm_yr_wk', 'weekday', 'wday', 'month', 'year', 'd',
       'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
       'snap_CA', 'snap_TX', 'snap_WI'],
      dtype='object')

In [25]:
calendar['month'] 

0       1
1       1
2       1
3       2
4       2
5       2
6       2
7       2
8       2
9       2
10      2
11      2
12      2
13      2
14      2
15      2
16      2
17      2
18      2
19      2
20      2
21      2
22      2
23      2
24      2
25      2
26      2
27      2
28      2
29      2
       ..
1939    5
1940    5
1941    5
1942    5
1943    5
1944    5
1945    5
1946    5
1947    5
1948    5
1949    5
1950    6
1951    6
1952    6
1953    6
1954    6
1955    6
1956    6
1957    6
1958    6
1959    6
1960    6
1961    6
1962    6
1963    6
1964    6
1965    6
1966    6
1967    6
1968    6
Name: month, Length: 1969, dtype: int8